In [27]:
# Amazon Be
import os
import boto3
from langchain_community.chat_models import BedrockChat
from langchain_community.embeddings import BedrockEmbeddings

boto3.setup_default_session(
    aws_access_key_id = os.getenv("aws_access_key_id"),
    aws_secret_access_key= os.getenv("aws_secret_access_key"),
    region_name = os.getenv("us-east-1"),
    )
# Set LLM and embeddings
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

model_kwargs =  { 
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

model_id = "anthropic.claude-v2"

model = BedrockChat(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs
)

embeddings = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id="amazon.titan-embed-text-v1"
)

In [4]:
from langchain.document_loaders import UnstructuredURLLoader
urls = [
    "https://aws.amazon.com/bedrock/",
    "https://aws.amazon.com/bedrock/claude/",
    "https://aws.amazon.com/bedrock/faqs/",
    "https://aws.amazon.com/bedrock/agents/",
    "https://aws.amazon.com/bedrock/developer-experience/",
    "https://aws.amazon.com/bedrock/guardrails/",
    "https://aws.amazon.com/bedrock/knowledge-bases/",
    "https://aws.amazon.com/bedrock/security-compliance/"
]
loader = UnstructuredURLLoader(urls=urls)
docs = loader.load()

[nltk_data] Downloading package punkt to C:\Users\Luan
[nltk_data]     Lima\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Luan Lima\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [16]:
for item in docs:
    print(item.page_content)
    print(item.metadata)

Artificial Intelligence›

Generative AI›

Amazon Bedrock

Amazon Bedrock

The easiest way to build and scale generative AI applications with foundation models

Get started with Amazon Bedrock

Try free demo

What is Amazon Bedrock?

Amazon Bedrock is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Mistral AI, Stability AI, and Amazon through a single API, along with a broad set of capabilities you need to build generative AI applications with security, privacy, and responsible AI. Using Amazon Bedrock, you can easily experiment with and evaluate top FMs for your use case, privately customize them with your data using techniques such as fine-tuning and Retrieval Augmented Generation (RAG), and build agents that execute tasks using your enterprise systems and data sources. Since Amazon Bedrock is serverless, you don't have to manage any infrastructure, and you can securely integrate

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
all_splits = text_splitter.split_documents(docs)

In [12]:
from langchain_community.vectorstores import FAISS

docsearch = FAISS.from_documents(all_splits, embeddings)
docsearch.save_local("faiss_index")
vectorstore= FAISS.load_local("faiss_index", embeddings)


In [28]:
# LCEL RAG Chain 🔗
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

# Retrieve and Generate
retriever = vectorstore.as_retriever(
    search_type="mmr", # Also test "similarity
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

question = "How does Amazon Bedrock help you keep your data and applications secure and private?"

# Chain Invoke
response = chain.invoke(question)
print(response)


 Based on the provided documents, Amazon Bedrock helps keep data and applications secure and private in the following ways:

- Data is encrypted in transit and at rest.

- Users can manage encryption keys using AWS Key Management Service (AWS KMS). 

- Identity-based policies provide control over data, managing what actions users can perform on resources.

- Data is not used to improve Amazon Bedrock's base models or shared with model providers. 

- Private connectivity can be established between Amazon Bedrock and Amazon VPC using AWS PrivateLink without exposing traffic to the internet.

- Amazon Bedrock complies with security and privacy standards like SOC, ISO, HIPAA, GDPR, and has CSA STAR Level 2 certification.

So in summary, encryption, access controls, compliance certifications, and private networking help keep data and applications secure with Amazon Bedrock.


In [29]:
retriever = vectorstore.as_retriever()

from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=model, retriever=retriever)

response = chain.invoke(question)
print(response['result'])

 Here are a few ways Amazon Bedrock helps keep data and applications secure and private:

- Encryption in transit and at rest - Your data is encrypted when moving over the network and when stored in Amazon Bedrock.

- AWS Key Management Service - You can create, manage, and control the encryption keys used to protect your data. 

- Identity policies - These allow you to control what actions different users and roles can perform on resources.

- Private connectivity - You can use AWS PrivateLink to establish private connectivity between your VPC and Bedrock without exposing your VPC to the internet.

- Compliance standards - Bedrock meets common compliance standards like ISO, SOC, HIPAA eligibility, and can be used in compliance with GDPR.

- Data isolation - When you tune a model, it uses a private copy so your data is not shared with model providers or used to improve base models. 

- Auditability - Bedrock provides capabilities to support security and compliance requirements like dat

In [ ]:
https://medium.com/@dminhk/langchain-question-answering-with-amazon-bedrock-1e6519735633